출처입니다.  
작업형 1 : https://www.kaggle.com/code/agileteam/4th-type1-python/notebook  
작업형 2 : https://www.kaggle.com/code/agileteam/4th-t2-python/notebook

In [6]:
import pandas as pd
import numpy as np
import warnings
import math

warnings.filterwarnings("ignore")

### 작업형 1

In [9]:
# 1-1. age 컬럼의 3사분위수와 1사분위수의 차를 절대값으로 구하고, 소수점 버려서, 정수로 출력

df = pd.read_csv("./data/csv/basic1.csv")

# df.info()
# df.head()

age_1q = df["age"].quantile(0.25)
age_3q = df["age"].quantile(0.75)

print("age 컬럼의 1분위수 : ",age_1q)
print("age 컬럼의 3분위수 : ",age_3q)

diff = abs(age_3q - age_1q)

print("두 값 차이의 절대값 =", int(diff))

age 컬럼의 1분위수 :  26.875
age 컬럼의 3분위수 :  77.0
두 값 차이의 절대값 = 50


In [11]:
# 1-2.(loves반응+wows반응)/(reactions반응) 비율이 0.4보다 크고 0.5보다 작으면서, type 컬럼이 'video'인 데이터의 갯수

df2 = pd.read_csv("./data/csv/fb.csv")

# df2.info()
# df2.head()

df2["ratio"] = (df2["loves"] + df2["wows"]) / df2["reactions"]

df2_cnt = df2[(df2["ratio"] > 0.4) & (df2["ratio"] < 0.5) & (df2["type"] == "video")]

print("데이터 개수 =", df2_cnt.shape[0])
print(df2_cnt.head())

데이터 개수 = 90
        id   type  reactions  comments  shares  likes  loves  wows  hahas  \
3116  3117  video        255       926     181    142    108     0      5   
4624  4625  video       1397       771     695    765    485   139      4   
4776  4777  video        259       695     263    151    106     0      2   
4793  4794  video        146       303     221     73     69     2      1   
4812  4813  video        184       530     224    101     80     0      1   

      sads  angrys     ratio  
3116     0       0  0.423529  
4624     1       3  0.446671  
4776     0       0  0.409266  
4793     0       1  0.486301  
4812     1       1  0.434783  


In [32]:
# 1-3. date_added가 2018년 1월 이면서 country가 United Kingdom 단독 제작인 데이터의 갯수

df3 = pd.read_csv("./data/csv/nf.csv")

# df3.info()
# df3.head()

# df3["country"].value_counts()
# df3["date_added"].unique()

df3_cond = df3[df3["date_added"].str.contains("January.*2018", na=False, regex=True) & (df3["country"]=="United Kingdom")]
# na=False는 null값을 False로 처리, regex=True는 정규 표현식으로 처리

print("조건을 만족하는 데이터 갯수 =",df3_cond.shape[0])

조건을 만족하는 데이터 갯수 = 6


### 작업형 2

In [66]:
# [마케팅] 자동차 시장 세분화
# 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
# 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!

# 예측할 값(y): "Segmentation" (1,2,3,4)
# 평가: Macro f1-score
# data: train.csv, test.csv

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

rdst = 22

car = pd.read_csv("./data/csv/train_4회유형.csv")   # 학습용 데이터

test = pd.read_csv("./data/csv/test_4회유형.csv")   # 테스트용 데이터

len_car = car.shape[0]


car_conc = pd.concat([car.drop("Segmentation", axis=1), test], axis=0)   # 학습용과 테스트 concat >> onehotencoding을 위해

car_conc.drop("ID", axis=1, inplace=True)   #ID 변수 제거


car_oh = pd.get_dummies(car_conc)   # 범주형 변수 onehotencoding

# train, test를 다시 분리

car_oh_tr = car_oh.iloc[:len_car]
car_oh_tt = car_oh.iloc[len_car:]

X = car_oh_tr.copy()
y = car["Segmentation"]

## train_test_split ##

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=rdst)

## MinMaxScaling ##

mmsc = MinMaxScaler()

mmsc.fit(X_train)

X_train_sc = mmsc.transform(X_train)
X_test_sc = mmsc.transform(X_test)

print(X_train_sc.shape, X_test_sc.shape)

## lightgbm ## 

lg = LGBMClassifier(n_estimators=100)

lg.fit(X_train_sc, y_train)

y_pred = lg.predict(X_test)

print("f1 score :", round(f1_score(y_test, y_pred, average="macro"),4))

## 교차 검증 ##
lg_g = LGBMClassifier()

params = {
    "num_leaves" : [10,20,30,50],
    "max_depth" : [3,5,7],
    "n_estimators" : [100, 200]
}

skfold = StratifiedKFold(shuffle=True,random_state=rdst)

gridcv = GridSearchCV(lg_g, param_grid=params, cv=skfold)

gridcv.fit(X_train_sc, y_train)

print("교차 검증 best score :", gridcv.best_score_)
print("교차 검증 best params :", gridcv.best_params_)

y_pred_grid = gridcv.predict(X_test_sc)

print("f1 score :", round(f1_score(y_test, y_pred_grid, average="macro"),4))


# test_data 예측

test_sc = mmsc.transform(car_oh_tt)

predict_test = gridcv.predict(test_sc)

print("분류 예측 값 :", predict_test)

(4665, 28) (2000, 28)
f1 score : 0.3835
교차 검증 best score : 0.5279742765273312
교차 검증 best params : {'max_depth': 3, 'n_estimators': 100, 'num_leaves': 10}
f1 score : 0.5441
분류 예측 값 : [1 3 3 ... 1 2 1]
